In [23]:
import pandas as pd
import numpy as np
import nltk

In [24]:
df = pd.read_csv('/songdata.csv')
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [25]:
df.shape

(57650, 4)

In [26]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \nLet the angels fly l...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \nMore power \nPower to...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \nis something i'll believe \nf...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \nam i frightened \nwhere can ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \nmake yourself at home \ni'm a bit ...


In [27]:
df.shape

(57650, 4)

In [28]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [29]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [30]:
df.head(10)

,artist,song,text
0,Cyndi Lauper,Carey,The wind is in from africa \nLast night I cou...
1,Loretta Lynn,I Know How,Yeah I love him like he wants me to and I know...
2,Tom T. Hall,And I Love You So,And I love you so the people ask me how \nHow...
3,Cliff Richard,Christmas Never Comes,"Talk about the heart of christmas, \nTalk abo..."
4,Loretta Lynn,Lead Me On,Once I made a promise that I'd never lead you ...
5,Iggy Pop,To Belong,A bird is sitting on the pavement \nSomeone b...
6,Counting Crows,Mercury,She is trapped inside a month of grey \nAnd t...
7,Electric Light Orchestra,Train Of Gold,"I saw her there from across the street, she wa..."
8,Morrissey,In The Future When All's Well,Armed with wealth and good health \nThe best ...
9,Vonda Shepard,The Radical Light,He was a poor boy having a hard time \nA shou...


In [31]:
df['text'][0]

"The wind is in from africa  \nLast night I couldn't sleep  \nOh, you know it sure is hard to leave here carey  \nBut it's really not my home  \nMy fingernails are filthy, I got beach tar on my feet  \nAnd I miss my clean white linen and my fancy french cologne  \n  \nOh carey get out your cane  \nAnd I'll put on some silver  \nOh you're a mean old daddy, but I like you fine  \n  \nCome on down to the mermaid cafe and I will buy you a bottle of wine  \nAnd we'll laugh and toast to nothing and smash our empty glasses down  \nLet's have a round for these freaks and these soldiers  \nA round for these friends of mine  \nLet's have another round for the bright red devil  \nWho keeps me in this tourist town  \n  \nCome on, carey, get out your cane  \nI'll put on some silver  \nOh you're a mean old daddy, but I like you  \n  \nMaybe I'll go to amsterdam  \nOr maybe I'll go to rome  \nAnd rent me a grand piano and put some flowers 'round my room  \nBut let's not talk about fare-thee-wells now

In [32]:
df.shape

(5000, 3)

In [33]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [34]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [35]:
nltk.download('punkt')
!nltk download punkt


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Usage: nltk [OPTIONS] COMMAND [ARGS]...
Try 'nltk -h' for help.

Error: No such command 'download'.


In [36]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [37]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [38]:
similarity[0]
df[df['song'] == 'Crying Over You']

,artist,song,text


In [39]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])

    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)

    return songs

In [46]:

# Assuming df is your DataFrame and 'column_name' is the column you're interested in
print(df.shape)  # For DataFrame 'df'
print(len(df['text']))  # For Series 'column_name'



(5000, 3)
5000


In [53]:
def recommendation(song):
    #to get recommendations
    if df.empty:
        return "DataFrame is empty"
    else:
        #process the DataFrame
        return song


In [54]:
recommendation('Crying Over You')


'Crying Over You'

In [55]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))